# Why pandas?

> Pandas handles heterogeneous relational data *table* while NumPy handles homogeneous *array*

https://pandas.pydata.org/pandas-docs/stable/

# What should we expect from pandas?

1. Data structure (models) for *TABLE*
2. CRUD 表的增删改查 Create, Read, Update and Delete
3. [Ecosystem](https://pandas.pydata.org/pandas-docs/stable/ecosystem.html?highlight=ecosystem() 生态系统，上游numpy 下游机器学习

In [2]:
import pandas as pd
import numpy as np

# 1. Series and DataFrame

Two basic data structures in Pandas are `Series` and `DataFrame`. An additional data structure that we use frequently is `Index`.

## 1.1 [Series](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html)

One-dimensional ndarray with axis labels (including time series).
- 一维 ndarray
- 带索引

```python

class Series(base.IndexOpsMixin, generic.NDFrame):
    """
    One-dimensional ndarray with axis labels (including time series).
    Labels need not be unique but must be a hashable type. The object
    supports both integer- and label-based indexing and provides a host of
    methods for performing operations involving the index. Statistical
    methods from ndarray have been overridden to automatically exclude
    missing data (currently represented as NaN).
    Operations between Series (+, -, /, *, **) align values based on their
    associated index values-- they need not be the same length. The result
    index will be the sorted union of the two indexes.
    Parameters
    ----------
    data : array-like, Iterable, dict, or scalar value
        Contains data stored in Series.
        .. versionchanged:: 0.23.0
           If data is a dict, argument order is maintained for Python 3.6
           and later.
    index : array-like or Index (1d)
        Values must be hashable and have the same length as `data`.
        Non-unique index values are allowed. Will default to
        RangeIndex (0, 1, 2, ..., n) if not provided. If both a dict and index
        sequence are used, the index will override the keys found in the
        dict.
    dtype : str, numpy.dtype, or ExtensionDtype, optional
        Data type for the output Series. If not specified, this will be
        inferred from `data`.
        See the :ref:`user guide <basics.dtypes>` for more usages.
    copy : bool, default False
        Copy input data.
    """

    _metadata = []  # type: List[str]
    _accessors = {"dt", "cat", "str", "sparse"}
    _deprecations = (
        base.IndexOpsMixin._deprecations
        | generic.NDFrame._deprecations
        | frozenset(
            [
                "asobject",
                "compress",
                "valid",
                "ftype",
                "real",
                "imag",
                "put",
                "ptp",
                "nonzero",
            ]
        )
    )

    # Override cache_readonly bc Series is mutable
    hasnans = property(
        base.IndexOpsMixin.hasnans.func, doc=base.IndexOpsMixin.hasnans.__doc__
    )
    _data = None  # type: SingleBlockManager

    # ----------------------------------------------------------------------
    # Constructors

    def __init__(
        self, data=None, index=None, dtype=None, name=None, copy=False, fastpath=False
    ):
        pass
```


In [3]:
obj = pd.Series([4,7,5,3])

In [4]:
obj

0    4
1    7
2    5
3    3
dtype: int64

In [5]:
obj.values # value of pandas is np.array

array([4, 7, 5, 3], dtype=int64)

In [6]:
obj.index

RangeIndex(start=0, stop=4, step=1)

In [7]:
obj2 = pd.Series([4,7,-5,3], index=['d', 'b', 'a', 'c'])

In [8]:
obj2

d    4
b    7
a   -5
c    3
dtype: int64

In [9]:
obj2.index

Index(['d', 'b', 'a', 'c'], dtype='object')

In [10]:
obj2['a']

-5

In [11]:
obj2[2]

-5

In [12]:
# 整数索引
obj2.iat[2]

-5

In [13]:
# 索引
obj2.at['a']

-5

In [14]:
obj2['d'] = 777

In [15]:
obj2

d    777
b      7
a     -5
c      3
dtype: int64

In [16]:
obj2.at['d'] = 888

In [17]:
obj2

d    888
b      7
a     -5
c      3
dtype: int64

In [18]:
# 大于2的数值
obj2[obj2>2]

d    888
b      7
c      3
dtype: int64

### dtype matters

In [19]:
obj2['d'] = 'hello'

In [20]:
obj2

d    hello
b        7
a       -5
c        3
dtype: object

In [22]:
# 不支持 字符串与整数 比较
# TypeError: '>' not supported between instances of 'str' and 'int'
# obj2[obj2>2]

## 1.2 [DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html)
Two-dimensional size-mutable, potentially heterogeneous tabular data structure with labeled axes (rows and columns). Arithmetic operations align on both row and column labels. Can be thought of as a dict-like container for Series objects. The primary pandas data structure.

```python
class DataFrame(NDFrame):
    """
    Two-dimensional, size-mutable, potentially heterogeneous tabular data.
    Data structure also contains labeled axes (rows and columns).
    Arithmetic operations align on both row and column labels. Can be
    thought of as a dict-like container for Series objects. The primary
    pandas data structure.
    Parameters
    ----------
    data : ndarray (structured or homogeneous), Iterable, dict, or DataFrame
        Dict can contain Series, arrays, constants, or list-like objects.
        .. versionchanged:: 0.23.0
           If data is a dict, column order follows insertion-order for
           Python 3.6 and later.
        .. versionchanged:: 0.25.0
           If data is a list of dicts, column order follows insertion-order
           for Python 3.6 and later.
    index : Index or array-like
        Index to use for resulting frame. Will default to RangeIndex if
        no indexing information part of input data and no index provided.
    columns : Index or array-like
        Column labels to use for resulting frame. Will default to
        RangeIndex (0, 1, 2, ..., n) if no column labels are provided.
    dtype : dtype, default None
        Data type to force. Only a single dtype is allowed. If None, infer.
    copy : bool, default False
        Copy data from inputs. Only affects DataFrame / 2d ndarray input.
    See Also
    --------
    DataFrame.from_records : Constructor from tuples, also record arrays.
    DataFrame.from_dict : From dicts of Series, arrays, or dicts.
    DataFrame.from_items : From sequence of (key, value) pairs
        read_csv, pandas.read_table, pandas.read_clipboard.
    Examples
    --------
    Constructing DataFrame from a dictionary.
    >>> d = {'col1': [1, 2], 'col2': [3, 4]}
    >>> df = pd.DataFrame(data=d)
    >>> df
       col1  col2
    0     1     3
    1     2     4
    Notice that the inferred dtype is int64.
    >>> df.dtypes
    col1    int64
    col2    int64
    dtype: object
    To enforce a single dtype:
    >>> df = pd.DataFrame(data=d, dtype=np.int8)
    >>> df.dtypes
    col1    int8
    col2    int8
    dtype: object
    Constructing DataFrame from numpy ndarray:
    >>> df2 = pd.DataFrame(np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]),
    ...                    columns=['a', 'b', 'c'])
    >>> df2
       a  b  c
    0  1  2  3
    1  4  5  6
    2  7  8  9
    """

    @property
    def _constructor(self) -> Type["DataFrame"]:
        return DataFrame

    _constructor_sliced = Series  # type: Type[Series]
    _deprecations = NDFrame._deprecations | frozenset(
        ["from_items"]
    )  # type: FrozenSet[str]
    _accessors = set()  # type: Set[str]

    @property
    def _constructor_expanddim(self):
        raise NotImplementedError("Not supported for DataFrames!")

    # ----------------------------------------------------------------------
    # Constructors

    def __init__(
        self,
        data=None,
        index: Optional[Axes] = None,
        columns: Optional[Axes] = None,
        dtype: Optional[Dtype] = None,
        copy: bool = False,
    ):
        pass
```

### Create DataFrame

|Type|Notes|
|-|-|
|2D ndarray | A matrix of data, passing optional row and column labels|
|dict of arrays, lists, or tuples | Each sequence becomes a column in the DataFrame; all sequences must be the same length|
|NumPy structured/record array | Treated as the “dict of arrays” case |
|dict of Series | Each value becomes a column; indexes from each Series are unioned together to form the result’s row index if no explicit index is passed |
| dict of dicts | Each inner dict becomes a column; keys are unioned to form the row index as in the “dict of Series” case |
| List of dicts or Series | Each item becomes a row in the DataFrame; union of dict keys or Series indexes become the DataFrame’s column labels |
| List of lists or tuples | Treated as the “2D ndarray” case 
| Another DataFrame | The DataFrame’s indexes are used unless different ones are passed
| NumPy MaskedArray | Like the “2D ndarray” case except masked values become NA/missing in the DataFrame result

In [23]:
# 使用字典创建 DataFrame
data = {'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada', 'Nevada'],
        'year': [2000, 2001, 2002, 2001, 2002, 2003],
        'pop': [1.5, 1.7, 3.6, 2.4, 2.9, 3.2]}
df = pd.DataFrame(data)

In [24]:
df

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9
5,Nevada,2003,3.2


In [25]:
df.head()

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9


In [26]:
# 指定列顺序
df = pd.DataFrame(data, columns=['year', 'pop', 'state'])

In [27]:
df

,year,pop,state
0,2000,1.5,Ohio
1,2001,1.7,Ohio
2,2002,3.6,Ohio
3,2001,2.4,Nevada
4,2002,2.9,Nevada
5,2003,3.2,Nevada


In [28]:
# 指定列名和索引（可以有空列）
df = pd.DataFrame(data, 
               columns=['year', 'pop', 'state', 'debt'],
               index=['one', 'two', 'three', 'four','five', 'six'])

In [29]:
df

,year,pop,state,debt
one,2000,1.5,Ohio,NaN
two,2001,1.7,Ohio,NaN
three,2002,3.6,Ohio,NaN
four,2001,2.4,Nevada,NaN
five,2002,2.9,Nevada,NaN
six,2003,3.2,Nevada,NaN


In [31]:
# KeyError: 1
# df[1]

In [33]:
# TypeError: '(slice(None, None, None), 1)' is an invalid key
# df[:, 1]

In [34]:
# 所有行
df[:]

,year,pop,state,debt
one,2000,1.5,Ohio,NaN
two,2001,1.7,Ohio,NaN
three,2002,3.6,Ohio,NaN
four,2001,2.4,Nevada,NaN
five,2002,2.9,Nevada,NaN
six,2003,3.2,Nevada,NaN


In [35]:
# 前2行
df[:2]

,year,pop,state,debt
one,2000,1.5,Ohio,NaN
two,2001,1.7,Ohio,NaN


In [37]:
# TypeError: '(slice(None, 2, None), slice(None, 2, None))' is an invalid key
# df[:2, :2]

In [38]:
# 只能选行
df[:2][:2]

,year,pop,state,debt
one,2000,1.5,Ohio,NaN
two,2001,1.7,Ohio,NaN


In [39]:
# 行loc location
df.loc['one']

year     2000
pop       1.5
state    Ohio
debt      NaN
Name: one, dtype: object

In [41]:
# loc 参数1为 行
# KeyError: 'year'
# df.loc['year'] # loc reads first input as row label

In [42]:
# loc 所有行，'year'列
df.loc[:, 'year'] # use the second

one      2000
two      2001
three    2002
four     2001
five     2002
six      2003
Name: year, dtype: int64

In [43]:
# 列
df['year']

one      2000
two      2001
three    2002
four     2001
five     2002
six      2003
Name: year, dtype: int64

In [45]:
# 只能选列
# KeyError: 'one'
# df['one']

In [46]:
# 某行某列
df.at['one', 'year']

2000

In [47]:
# 整数索引的某行某列
df.iloc[1, 1]

1.7

In [48]:
# 第1行（从0开始）
df.iloc[1]

year     2001
pop       1.7
state    Ohio
debt      NaN
Name: two, dtype: object

In [49]:
# 第1列（从0开始）
df.iloc[:,1]

one      1.5
two      1.7
three    3.6
four     2.4
five     2.9
six      3.2
Name: pop, dtype: float64

In [50]:
# 给某列所有行赋值
df['debt'] = 16.5 # boardcast

In [51]:
df

,year,pop,state,debt
one,2000,1.5,Ohio,16.5
two,2001,1.7,Ohio,16.5
three,2002,3.6,Ohio,16.5
four,2001,2.4,Nevada,16.5
five,2002,2.9,Nevada,16.5
six,2003,3.2,Nevada,16.5


In [52]:
val = pd.Series([-1.2, -1.5, -1.7], index=['two', 'three', 'five'])

In [53]:
# 通过 Series 赋值
df['debt'] = val # assign by index

In [54]:
df

,year,pop,state,debt
one,2000,1.5,Ohio,NaN
two,2001,1.7,Ohio,-1.2
three,2002,3.6,Ohio,-1.5
four,2001,2.4,Nevada,NaN
five,2002,2.9,Nevada,-1.7
six,2003,3.2,Nevada,NaN


In [55]:
# 新增一列
df['eastern'] = (df.state == 'Ohio') # this is a way to add new columns 

In [56]:
df

,year,pop,state,debt,eastern
one,2000,1.5,Ohio,NaN,True
two,2001,1.7,Ohio,-1.2,True
three,2002,3.6,Ohio,-1.5,True
four,2001,2.4,Nevada,NaN,False
five,2002,2.9,Nevada,-1.7,False
six,2003,3.2,Nevada,NaN,False


In [57]:
# 转置
df.T # transpose is the same as numpy

,one,two,three,four,five,six
year,2000,2001,2002,2001,2002,2003
pop,1.5,1.7,3.6,2.4,2.9,3.2
state,Ohio,Ohio,Ohio,Nevada,Nevada,Nevada
debt,NaN,-1.2,-1.5,NaN,-1.7,NaN
eastern,True,True,True,False,False,False


In [58]:
type(df['pop'])

pandas.core.series.Series

In [59]:
df.index

Index(['one', 'two', 'three', 'four', 'five', 'six'], dtype='object')

## 1.3 [Index](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Index.html)
```python
class Index(IndexOpsMixin, PandasObject):
    """
    Immutable ndarray implementing an ordered, sliceable set. The basic object
    storing axis labels for all pandas objects.
    Parameters
    ----------
    data : array-like (1-dimensional)
    dtype : NumPy dtype (default: object)
        If dtype is None, we find the dtype that best fits the data.
        If an actual dtype is provided, we coerce to that dtype if it's safe.
        Otherwise, an error will be raised.
    copy : bool
        Make a copy of input ndarray.
    name : object
        Name to be stored in the index.
    tupleize_cols : bool (default: True)
        When True, attempt to create a MultiIndex if possible.
    See Also
    --------
    RangeIndex : Index implementing a monotonic integer range.
    CategoricalIndex : Index of :class:`Categorical` s.
    MultiIndex : A multi-level, or hierarchical, Index.
    IntervalIndex : An Index of :class:`Interval` s.
    DatetimeIndex, TimedeltaIndex, PeriodIndex
    Int64Index, UInt64Index,  Float64Index
    Notes
    -----
    An Index instance can **only** contain hashable objects
    Examples
    --------
    >>> pd.Index([1, 2, 3])
    Int64Index([1, 2, 3], dtype='int64')
    >>> pd.Index(list('abc'))
    Index(['a', 'b', 'c'], dtype='object')
    """
```

## 1.4 Composed Pandas Data Structure [pandas array](https://pandas.pydata.org/pandas-docs/stable/reference/arrays.html)

# 2 Create, Read, Update and Delete : CRUD

CRUD of table

|Type of operation|Functions|
|-|-|
|Create|Create from file (e.g. csv file); Create from memory (e.g. Constructor);, handling different data types ...|
|Read|Indexing and Selection; Groupby; Functions ...|
|Update|Merge, join and concatenate; Reshape ...|
|Delete|drop duplicates ...|

We focus on three topics
1. Advanced DataFrame Functionality
    1. data selection
    1. data alignment
    1. function application
1. Missing Values & Transform
1. Merge, Join and Concatinate
1. Groupby = split + apply + combine

## 2.1 Advanced DataFrame Functionality

In [60]:
data = pd.DataFrame(np.random.randn(4, 4),
                    index=['Beijing', 'Shanghai', 'Guangzhou', 'Shenzhen'],
                    columns=['one', 'two', 'three', 'four'])

In [61]:
data

,one,two,three,four
Beijing,0.384187,-0.945130,0.537665,0.306934
Shanghai,-0.435012,0.576906,-0.683838,-0.746249
Guangzhou,0.421750,-0.982795,-0.388342,0.993025
Shenzhen,0.898462,0.508325,0.121854,-1.345726


In [62]:
# 删除行
data.drop('Beijing')

,one,two,three,four
Shanghai,-0.435012,0.576906,-0.683838,-0.746249
Guangzhou,0.421750,-0.982795,-0.388342,0.993025
Shenzhen,0.898462,0.508325,0.121854,-1.345726


In [63]:
## 删除行
data.drop(['Shanghai', 'Shenzhen'])

,one,two,three,four
Beijing,0.384187,-0.945130,0.537665,0.306934
Guangzhou,0.421750,-0.982795,-0.388342,0.993025


In [64]:
## 删除列
data.drop('one', axis=1)

,two,three,four
Beijing,-0.945130,0.537665,0.306934
Shanghai,0.576906,-0.683838,-0.746249
Guangzhou,-0.982795,-0.388342,0.993025
Shenzhen,0.508325,0.121854,-1.345726


In [65]:
# Series
s1 = pd.Series(np.arange(3), index=['a', 'b', 'c'])
s2 = pd.Series(np.arange(3), index=['c', 'd', 'e'])

In [66]:
s1+s2

a    NaN
b    NaN
c    2.0
d    NaN
e    NaN
dtype: float64

In [67]:
# Series 填空值
s1.add(s2, fill_value=0)

a    0.0
b    1.0
c    2.0
d    1.0
e    2.0
dtype: float64

In [68]:
df1 = pd.DataFrame(np.random.randn(3, 3), columns=list('bcd'), index=list('123'))
df2 = pd.DataFrame(np.random.randn(3, 3), columns=list('ade'), index=list('369'))

In [69]:
df1+df2

,a,b,c,d,e
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,0.784325,NaN
6,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN


In [70]:
df1.add(df2)

,a,b,c,d,e
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,0.784325,NaN
6,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN


In [71]:
# DataFrame 填空行，只有原来有值的才填
df1.radd(df2, fill_value=1)

,a,b,c,d,e
1,NaN,2.596167,-0.073671,1.107000,NaN
2,NaN,0.807469,2.214203,-0.121378,NaN
3,1.651452,0.573101,1.094697,0.784325,2.052481
6,2.100015,NaN,NaN,1.421038,1.971556
9,-0.365213,NaN,NaN,1.481185,1.130697


In [72]:
df1 + np.ones((3,3))

,b,c,d
1,2.596167,-0.073671,1.107000
2,0.807469,2.214203,-0.121378
3,0.573101,1.094697,1.041437


In [73]:
data

,one,two,three,four
Beijing,0.384187,-0.945130,0.537665,0.306934
Shanghai,-0.435012,0.576906,-0.683838,-0.746249
Guangzhou,0.421750,-0.982795,-0.388342,0.993025
Shenzhen,0.898462,0.508325,0.121854,-1.345726


In [74]:
np.abs(data)

,one,two,three,four
Beijing,0.384187,0.945130,0.537665,0.306934
Shanghai,0.435012,0.576906,0.683838,0.746249
Guangzhou,0.421750,0.982795,0.388342,0.993025
Shenzhen,0.898462,0.508325,0.121854,1.345726


### apply func to one dimension a pd.DataFrame

In [75]:
f = lambda x: x.max() - x.min()
data.apply(f)

one      1.333474
two      1.559701
three    1.221502
four     2.338751
dtype: float64

In [76]:
data.apply(f, axis=1)

Beijing      1.482795
Shanghai     1.323155
Guangzhou    1.975820
Shenzhen     2.244187
dtype: float64

In [77]:
fs = lambda x: pd.Series([x.min(), x.max(), f(x)], index=['min', 'max', 'range'])

In [78]:
# 默认按行计算
data.apply(fs)

,one,two,three,four
min,-0.435012,-0.982795,-0.683838,-1.345726
max,0.898462,0.576906,0.537665,0.993025
range,1.333474,1.559701,1.221502,2.338751


In [79]:
# 按列计算
data.apply(fs, axis='columns')

,min,max,range
Beijing,-0.945130,0.537665,1.482795
Shanghai,-0.746249,0.576906,1.323155
Guangzhou,-0.982795,0.993025,1.975820
Shenzhen,-1.345726,0.898462,2.244187


### map func to each element in a pd.Series

In [80]:
s = pd.Series(np.random.randn(10), index=np.arange(10))

In [81]:
s

0   -0.299850
1    0.890025
2   -1.537049
3    1.684907
4   -0.400826
5   -1.136783
6    0.305228
7    0.697341
8   -1.849488
9    0.292730
dtype: float64

In [82]:
# map 对Series每一个元素计算
import math
s.map(lambda x: x - math.floor(x))

0    0.700150
1    0.890025
2    0.462951
3    0.684907
4    0.599174
5    0.863217
6    0.305228
7    0.697341
8    0.150512
9    0.292730
dtype: float64

### applymap func to each elements in a pd.DataFrame

In [85]:
df = pd.DataFrame(np.random.randn(3,3))
df

,0,1,2
0,-0.102601,0.147842,0.229289
1,1.548299,-0.306837,0.312332
2,0.681730,-0.586444,0.081211


In [86]:
df.describe()

,0,1,2
count,3.000000,3.000000,3.000000
mean,0.709143,-0.248480,0.207611
std,0.825792,0.370605,0.117075
min,-0.102601,-0.586444,0.081211
25%,0.289564,-0.446640,0.155250
50%,0.681730,-0.306837,0.229289
75%,1.115015,-0.079498,0.270810
max,1.548299,0.147842,0.312332


In [87]:
# map 对每一个元素计算
df.applymap(lambda x: "{:.4E}".format(x)).describe()

,0,1,2
count,3,3,3
unique,3,3,3
top,-1.0260E-01,-3.0684E-01,2.2929E-01
freq,1,1,1


## 2.2 Data Transform

### 2.2.1 Missing Data

We provide two kinds of ways to handle the missing data
1. filter out
2. fill in

In [91]:
data = pd.DataFrame([[1, 2, 4],
                    [np.nan, np.nan, 2],
                    [np.nan, 3, 3]])
data

,0,1,2
0,1.0,2.0,4
1,NaN,NaN,2
2,NaN,3.0,3


In [92]:
data.describe()

,0,1,2
count,1.0,2.000000,3.0
mean,1.0,2.500000,3.0
std,NaN,0.707107,1.0
min,1.0,2.000000,2.0
25%,1.0,2.250000,2.5
50%,1.0,2.500000,3.0
75%,1.0,2.750000,3.5
max,1.0,3.000000,4.0


In [93]:
data.dropna()

,0,1,2
0,1.0,2.0,4


In [94]:
data.dropna(axis=1)

,2
0,4
1,2
2,3


In [95]:
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,2.0,4
1,NaN,NaN,2
2,NaN,3.0,3


In [96]:
# 一行中有3个可信数据才保留
data.dropna(thresh=3)

,0,1,2
0,1.0,2.0,4


In [97]:
data.dropna(thresh=2)

,0,1,2
0,1.0,2.0,4
2,NaN,3.0,3


In [98]:
data.dropna(thresh=1)

,0,1,2
0,1.0,2.0,4
1,NaN,NaN,2
2,NaN,3.0,3


In [99]:
# 填充
data.fillna('10')

,0,1,2
0,1,2,4
1,10,10,2
2,10,3,3


In [100]:
data.fillna({0: 0, 1: 1})

,0,1,2
0,1.0,2.0,4
1,0.0,1.0,2
2,0.0,3.0,3


In [101]:
# 找 forward
data.fillna(method='ffill')

,0,1,2
0,1.0,2.0,4
1,1.0,2.0,2
2,1.0,3.0,3


In [102]:
# 找 backward
data.fillna(method='bfill')

,0,1,2
0,1.0,2.0,4
1,NaN,3.0,2
2,NaN,3.0,3


### 2.2.2 Transform

In [103]:
data = pd.DataFrame({"k1": ['one', 'two'] * 3 + ['two'],
                     "k2": [1,1,2,3,3,4,4]})

In [104]:
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [105]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [106]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [107]:
data.replace('one', 'three')

,k1,k2
0,three,1
1,two,1
2,three,2
3,two,3
4,three,3
5,two,4
6,two,4


In [108]:
data.columns = data.columns.map(lambda x: x + "_column_name")

In [109]:
data

,k1_column_name,k2_column_name
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [110]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
bins = [18, 25, 35, 60, 100]
cuts = pd.cut(ages, bins)

In [111]:
cuts

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [112]:
cuts.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [113]:
# pandas.core.arrays.categorical.Categorical
type(cuts)

pandas.core.arrays.categorical.Categorical

In [114]:
cuts.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

In [115]:
pd.value_counts(cuts)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

In [116]:
s = pd.Series(np.random.randn(10)*10)

In [117]:
s

0    -2.000187
1    -1.271868
2    13.834755
3   -11.860642
4    -7.175217
5     6.234651
6    -4.485044
7   -14.127924
8     2.791582
9    -6.143641
dtype: float64

In [118]:
# 按百分数cut
pd.qcut(s, q=4, labels=['a', 'b', 'c', 'd'])

0    c
1    c
2    d
3    a
4    a
5    d
6    b
7    a
8    d
9    b
dtype: category
Categories (4, object): [a < b < c < d]

## 2.3 Data Wrangling

### 2.3.1 Join data

In [122]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [123]:
df2 = pd.DataFrame({'key': ['b', 'a', 'e'], 'data2': range(3)})
df2

,key,data2
0,b,0
1,a,1
2,e,2


In [124]:
# 内连接 共有key的数据
pd.merge(df1, df2, how='inner')

,key,data1,data2
0,b,0,0
1,b,1,0
2,b,6,0
3,a,2,1
4,a,4,1
5,a,5,1


In [125]:
# 外连接 所有key的数据
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0.0,0.0
1,b,1.0,0.0
2,b,6.0,0.0
3,a,2.0,1.0
4,a,4.0,1.0
5,a,5.0,1.0
6,c,3.0,NaN
7,e,NaN,2.0


In [126]:
# 左外连接 左边表的key
pd.merge(df1, df2, how='left')

,key,data1,data2
0,b,0,0.0
1,b,1,0.0
2,a,2,1.0
3,c,3,NaN
4,a,4,1.0
5,a,5,1.0
6,b,6,0.0


In [127]:
# 右外连接 右边表的key
pd.merge(df1, df2, how='right')

,key,data1,data2
0,b,0.0,0
1,b,1.0,0
2,b,6.0,0
3,a,2.0,1
4,a,4.0,1
5,a,5.0,1
6,e,NaN,2


In [128]:
# 表的拼接，默认join='outer'
# 默认按行拼接，不会像merge一样去另一张表找相同key的数据填充，而是直接填NaN
pd.concat([df1, df2])

C:\Anaconda3\envs\python3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,data1,data2,key
0,0.0,NaN,b
1,1.0,NaN,b
2,2.0,NaN,a
3,3.0,NaN,c
4,4.0,NaN,a
5,5.0,NaN,a
6,6.0,NaN,b
0,NaN,0.0,b
1,NaN,1.0,a
2,NaN,2.0,e


## 2.3 Split Apply Combine
![image.png](attachment:image.png)

In [129]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})

In [130]:
df

,key1,key2,data1,data2
0,a,one,1.140618,1.064196
1,a,two,1.329632,0.049261
2,b,one,0.961739,-0.090824
3,b,two,0.529905,-0.413152
4,a,one,0.454355,-0.829439


In [131]:
grouped = df['data1'].groupby(df['key1'])

In [132]:
grouped.mean()

key1
a    0.974868
b    0.745822
Name: data1, dtype: float64

In [133]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()

In [134]:
means

key1  key2
a     one     0.797486
      two     1.329632
b     one     0.961739
      two     0.529905
Name: data1, dtype: float64

In [135]:
means.unstack()

key2,one,two
key1,,
a,0.797486,1.329632
b,0.961739,0.529905


In [136]:
for name, group in df.groupby('key1'):  ## could be understood as [(name, group)]
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  1.140618  1.064196
1    a  two  1.329632  0.049261
4    a  one  0.454355 -0.829439
b
  key1 key2     data1     data2
2    b  one  0.961739 -0.090824
3    b  two  0.529905 -0.413152


In [137]:
for name, group in df.groupby(['key1', 'key2']):  ## could be understood as [(name, group)]
    print(name)
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  1.140618  1.064196
4    a  one  0.454355 -0.829439
('a', 'two')
  key1 key2     data1     data2
1    a  two  1.329632  0.049261
('b', 'one')
  key1 key2     data1     data2
2    b  one  0.961739 -0.090824
('b', 'two')
  key1 key2     data1     data2
3    b  two  0.529905 -0.413152


In [138]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [139]:
for n, d in df.groupby(df.dtypes, axis=1):
    print(n)
    print(d)

float64
      data1     data2
0  1.140618  1.064196
1  1.329632  0.049261
2  0.961739 -0.090824
3  0.529905 -0.413152
4  0.454355 -0.829439
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


In [140]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.117379
     two   0.049261
b    one  -0.090824
     two  -0.413152

In [141]:
df.groupby(['key1', 'key2'])['data2'].mean()

key1  key2
a     one     0.117379
      two     0.049261
b     one    -0.090824
      two    -0.413152
Name: data2, dtype: float64

In [142]:
test_df = pd.DataFrame(np.concatenate([np.random.randn(100, 10), 
                                       np.random.randint(0, 3, (100,1))],
                                       axis=1), 
                       columns=['key{}'.format(i) for i in range(11)])

def top(df, n=5, column='key0'):
    return df.sort_values(by=column)[-n:]

test_df

,key0,key1,key2,key3,key4,key5,key6,key7,key8,key9,key10
0,0.759262,-0.438232,-0.237963,0.321822,-1.959240,0.242953,-0.721777,1.067455,0.076349,1.123335,2.0
1,1.215052,1.289028,-1.143655,0.447554,-0.696834,0.056494,-0.390488,0.350158,-1.079518,-0.585421,1.0
2,0.448901,1.571796,1.779308,-0.389856,2.401280,0.033169,-1.616395,1.981655,-0.700640,-1.289937,1.0
3,1.201925,-0.425031,-0.666898,-0.626180,0.451156,1.573338,-1.881986,0.093342,0.323530,-1.219871,2.0
4,-0.260729,0.615146,-0.096646,-1.212925,-0.365106,0.358676,0.307381,0.301608,-0.813167,0.322085,0.0
...,...,...,...,...,...,...,...,...,...,...,...
95,0.503689,-0.122156,1.632682,-0.055488,1.714351,1.648674,0.605374,-0.108246,2.252532,-0.466417,0.0
96,-0.973557,0.639049,1.399896,-2.474654,0.415843,-0.561875,1.332863,-1.122244,0.864680,-0.482171,1.0
97,-1.165662,-0.419372,-1.199017,0.118661,0.267361,-0.389914,1.499606,-1.407092,-0.410580,1.607209,1.0
98,-0.469414,1.469967,0.009962,0.013327,-1.185294,0.372822,-0.529233,-1.602278,0.040712,-1.489256,2.0


In [143]:
test_df.groupby('key10').apply(top)

key0      key1      key2      key3      key4      key5  \
key10                                                                  
0.0   40  0.630165  0.406276 -0.032949  2.569666 -1.976348 -0.367629   
      77  0.650953 -1.164661 -0.357382  1.340775 -0.191205 -0.246360   
      59  0.665016  1.151981 -2.045111  0.402626  0.353777  0.671671   
      69  1.161067 -0.253604  0.612615  1.046471  1.716613 -0.547228   
      11  1.822529 -0.497001  0.291409  0.741100  0.766884  0.563576   
1.0   83  0.822760  0.406358 -0.894374  0.825807  0.650872  1.317012   
      38  1.001183 -2.379814  0.950209  0.775898 -0.125922 -1.609237   
      1   1.215052  1.289028 -1.143655  0.447554 -0.696834  0.056494   
      33  1.326173 -0.207377  0.021734  1.200513  0.547203 -0.492572   
      45  1.967864 -0.280515  1.214957  0.768051  0.546035  0.146497   
2.0   39  1.275298 -2.743741 -1.238485  1.098494 -1.180497  0.135574   
      87  1.384606  1.125660  0.286646 -0.540985 -0.444860 -0.871666   
      36  1.448369  0.366343  0.416347  0.078111 -0.641759 -0.150414   
      79  1.931450  1.196936  0.647495  0.304149  1.486258  0.217246   
      30  2.341104  0.028574  1.067579 -0.374351 -1.164676  0.361177   

              key6      key7      key8      key9  key10  
key10                                                    
0.0   40  0.118870 -0.650967 -2.599879 -0.084730    0.0  
      77 -0.630138 -0.239213  1.903388 -0.893006    0.0  
      59 -1.465207 -0.530004 -0.972909 -0.162447    0.0  
      69  1.802996 -0.767496  1.417925  0.496476    0.0  
      11  0.633504  0.001837  1.177661  0.485625    0.0  
1.0   83  0.154774 -0.217571  0.511240 -2.182462    1.0  
      38  0.202209 -0.747152  0.298431  0.617184    1.0  
      1  -0.390488  0.350158 -1.079518 -0.585421    1.0  
      33  1.325668 -1.124620  0.873322 -0.226515    1.0  
      45 -1.508948  0.877820  0.177469  0.197622    1.0  
2.0   39 -0.759182 -0.840666  0.297053 -0.492689    2.0  
      87  1.828482  1.746045  0.315082 -1.323005    2.0  
      36  0.605531  0.853011 -1.164340 -1.453595    2.0  
      79  0.597722 -1.148517 -0.548962  1.894884    2.0  
      30  0.555026  0.348657 -1.676694  2.165981    2.0